In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

In [2]:
test0 = ['###############',
         '#.......#....E#',
         '#.#.###.#.###.#',
         '#.....#.#...#.#',
         '#.###.#####.#.#',
         '#.#.#.......#.#',
         '#.#.#####.###.#',
         '#...........#.#',
         '###.#.#####.#.#',
         '#...#.....#.#.#',
         '#.#.#.###.#.#.#',
         '#.....#...#.#.#',
         '#.###.#.#.#.#.#',
         '#S..#.....#...#',
         '###############']

test1 = ['#################',
         '#...#...#...#..E#',
         '#.#.#.#.#.#.#.#.#',
         '#.#.#.#...#...#.#',
         '#.#.#.#.###.#.#.#',
         '#...#.#.#.....#.#',
         '#.#.#.#.#.#####.#',
         '#.#...#.#.#.....#',
         '#.#.#####.#.###.#',
         '#.#.#.......#...#',
         '#.#.###.#####.###',
         '#.#.#...#.....#.#',
         '#.#.#.#####.###.#',
         '#.#.#.........#.#',
         '#.#.#.#########.#',
         '#S#.............#',
         '#################']

In [3]:
def get_graph(data):
    graph = []
    for line in data:
        line = line.strip()
        graph.append(list(line))
    graph = np.array(graph)
    
    start = np.where(graph == 'S')
    start = (start[0][0],start[1][0])
    
    end = np.where(graph == 'E')
    end = (end[0][0],end[1][0])
    
    return graph, start, end

def BFS(graph, start, end):
    dxdy = [[-1,0],[0,1],[1,0],[0,-1]]
    
    start = (start[0], start[1], 1)
    queue = deque([start])
    dist = {start:0}
    
    while len(queue):
        cur_pos = queue.pop()
        cur_xy = (cur_pos[0], cur_pos[1])
        cur_dir = cur_pos[2]
        
        cur_dst = dist[cur_pos]
        
        for i in range(-1, 2):
            nxt_dir = cur_dir+i
            if nxt_dir >= len(dxdy):
                nxt_dir -= len(dxdy)
            elif nxt_dir < 0:
                nxt_dir += len(dxdy)
                
            xy = dxdy[nxt_dir]
            nxt_xy = (cur_xy[0]+xy[0], cur_xy[1]+xy[1])
            if graph[nxt_xy[0],nxt_xy[1]] == '#':
                continue
                
            nxt_dst = cur_dst+1
            if nxt_dir != cur_dir:
                nxt_dst += 1000
                
            nxt_pos = (nxt_xy[0], nxt_xy[1], nxt_dir)
            if nxt_pos in dist.keys():
                if dist[nxt_pos] > nxt_dst:
                    dist[nxt_pos] = nxt_dst
                    if nxt_xy != end:
                        queue.append(nxt_pos)
            else:
                dist[nxt_pos] = nxt_dst
                if nxt_xy != end:
                    queue.append(nxt_pos)
                
    return dist

def end_score(dist, end):
    best = 1e6
    for i in range(0, 4):
        if (end[0],end[1],i) in dist.keys():
            if dist[(end[0],end[1],i)] < best:
                best = dist[(end[0],end[1],i)]
    return best

def DFS(dist, end, cur_pos, cur_score, cur_path, path):   
    dxdy = [[-1,0],[0,1],[1,0],[0,-1]]
    
    cur_path.append((cur_pos[0], cur_pos[1]))
    if cur_path[-1] == end:
        if cur_score == 0:
            for p in cur_path:
                if p not in path:
                    path.append(p)
        else:
            return path
    
    cur_dir = cur_pos[2]
    xy = dxdy[cur_dir]
    
    tmp = (cur_pos[0], cur_pos[1])
    
    for nxt_dir in range(0, len(dxdy)):
        if abs(nxt_dir-cur_dir) == 2:
            continue
            
        nxt_score = cur_score-1
        if nxt_dir != cur_dir:
            nxt_score -= 1000
            
        if nxt_score < 0:
            continue
        
        nxt_pos = (cur_pos[0]-xy[0], cur_pos[1]-xy[1], nxt_dir)
        if (cur_pos[0]-xy[0], cur_pos[1]-xy[1]) in cur_path:
            continue
        
        if nxt_pos not in dist.keys():
            continue
        elif dist[nxt_pos] != nxt_score:
            continue
        path = DFS(dist, end, nxt_pos, nxt_score, deepcopy(cur_path), path)
        
    return path

def clean_path(path):
    cln_path = []
    for pos in path:
        if pos not in cln_path:
            cln_path.append(pos)
    return cln_path

def print_graph(graph, path=None):
    for i in range(0, len(graph)):
        string = ''
        for j in range(0, len(graph[i])):
            if path is not None and (i,j) in path:
                string += 'O'
            else:
                string += graph[i][j]
        print(string)

def run(data, prnt=False):
    graph, start, end = get_graph(data)
    dist = BFS(graph, start, end)
    score = end_score(dist, end)
    print('Part 1 result:', score)
    
    path = []
    for i in range(0, 4):
        cur_pos = (end[0], end[1], i)
        if cur_pos in dist.keys() and dist[cur_pos] == score:
            path = DFS(dist, start, cur_pos, deepcopy(score), [], path)
    path = clean_path(path)
    
    print('Part 2 result:', len(path))
    if prnt:
        print_graph(graph, path)
    
run(test0, True)
print()
run(test1, True)

Part 1 result: 7036
Part 2 result: 45
###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############

Part 1 result: 11048
Part 2 result: 64
#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################


In [4]:
with open('input_day16.txt', 'r') as f:
    data = f.readlines()
    f.close()
run(data)

Part 1 result: 90440
Part 2 result: 479
